In [16]:
import json
import re
import parsivar
from collections import Counter
from string import punctuation
import numpy as np

In [17]:
with open('IR_data_news.json', 'r', encoding='utf-8') as file:
    json_data = file.read()
data = json.loads(json_data)
news = {}
contents = []
titles = []
urls =[]
for id, dt in data.items():
    news[str(id)] = { 'content': dt['content'],
                       'title': dt['title'],
                       'url': dt['url']
    }
    contents.append(dt['content'])
    titles.append(dt['title'])
    urls.append(dt['url'])

## Tokenize

In [18]:
def tokenize(text):
    pattern = re.compile(r'([؟!?]+|[\d.:]+|[:.،؛»\])}"«\[({/\\])')
    text = pattern.sub(r" \1 ", text.replace("\n", " ").replace("\t", " ").replace("\xa0"," "))
    tokens = [word for word in text.split(" ") if word]
    punctuations = [
        ')', '(', '>', '<', "؛", "،", '{', '}', "؟", ':', "–", '»', '"', '«', '[', ']', '"', '+', '=', '?',
        '/', '//', '\\', '|', '!', '%', '&', '*', '$', '#', '؟', '*', '.', '_', '', 'EMAIL', 'LINK', 'ID', 'NUM',
        'NUMF', ' ', '-'
    ]
    tokens = [element for element in tokens if element not in punctuations] 
    
    before_verbs = {
        "خواهم",
        "خواهی",
        "خواهد",
        "خواهیم",
        "خواهید",
        "خواهند",
        "نخواهم",
        "نخواهی",
        "نخواهد",
        "نخواهیم",
        "نخواهید",
        "نخواهند",
    }
    if len(tokens) == 1:
        return tokens
    result = [""]
    for token in reversed(tokens):
        if token in before_verbs:
            result[-1] = token + "_" + result[-1]
        else:
            result.append(token)
    return list(reversed(result[1:]))
    

            
text = 'خواهم رفت'
text2 = news['0']['content']
print(tokenize(text2))

['به', 'گزارش', 'خبرگزاری', 'فارس', 'کنفدراسیون', 'فوتبال', 'آسیا', 'AFC', 'در', 'نامه', 'ای', 'رسمی', 'به', 'فدراسیون', 'فوتبال', 'ایران', 'و', 'باشگاه', 'گیتی', 'پسند', 'زمان', 'قرعه', 'کشی', 'جام', 'باشگاه', 'های', 'فوتسال', 'آسیا', 'را', 'رسماً', 'اعلام', 'کرد', 'بر', 'این', 'اساس', '25', 'فروردین', 'ماه', '1401', 'مراسم', 'قرعه', 'کشی', 'جام', 'باشگاه', 'های', 'فوتسال', 'آسیا', 'در', 'مالزی', 'برگزار', 'می', 'شود', 'باشگاه', 'گیتی', 'پسند', 'بعنوان', 'قهرمان', 'فوتسال', 'ایران', 'در', 'سال', '1400', 'به', 'این', 'مسابقات', 'راه', 'پیدا', 'کرده', 'است', 'پیش', 'از', 'این', 'گیتی', 'پسند', 'تجربه', '3', 'دوره', 'حضور', 'در', 'جام', 'باشگاه', 'های', 'فوتسال', 'آسیا', 'را', 'داشته', 'که', 'هر', 'سه', 'دوره', 'به', 'فینال', 'مسابقات', 'راه', 'پیدا', 'کرده', 'و', 'یک', 'عنوان', 'قهرمانی', 'و', 'دو', 'مقام', 'دومی', 'بدست', 'آورده', 'است', 'انتهای', 'پیام']


## Normalization

In [19]:
class TextNormalizer:
    def __init__(self):
        self.setup_patterns()
        self.load_verb_data()

    def setup_patterns(self):
        self.punctuation_marks = {
            'after': r"\.:!،؛؟»\]\)\}",
            'before': r"«\[\(\{",
            'all': r"[\.:!،؛؟»\'\]\)\}|«\[\(\/\{><+\-?!=_]"
        }
        self.spacing_patterns = {
            'extra_spaces': (r" {2,}", " "),
            'extra_newlines': (r"\n{3,}", "\n\n"),
            'extra_ZWNJs': (r"\u200c{2,}", "\u200c"),
            'remove_ZWNJ_spaces': (r"\u200c+\s|\s+\u200c+", " "),
            'trim_ZWNJ_word_bounds': (r"\b\u200c*\B|\B\u200c*\b", " "),
            'remove_keshide': (r"[-\r]", " "),
            'remove_no_break_space': (r"\xa0", " "),  # remove no-break char
            'fix_ye_space': (r"([^ ]) ی ", r"\1‌ی "),  # fix 'ی' space
            'fix_mi_space': (r"(^| )(ن?می) ", r"\1\2‌"),  # fix 'می' and 'نمی' space
            'fix_suffix_spacing': (r"(?<=[^\n\d" + self.punctuation_marks['after'] + self.punctuation_marks['before'] + r"]{2}) (تر(ین?)?|گری?|های?)(?=[ \n" + self.punctuation_marks['after'] + self.punctuation_marks['before'] + r"]|$)", r"‌\1"),  # fix suffix spacing
            'fix_verb_conjugation_spacing': (r"([^ ]ه) (ا(م|یم|ش|ند|ی|ید|ت))(?=[ \n" + self.punctuation_marks['after'] + r"]|$)", r"\1‌\2"),  # fix verb conjugation spacing
             'fix_ha_suffix': (r"(ه)(ها)", r"\1‌\2"),  # fix 'ها' suffix
             'remove_quotation_spaces': ('" ([^\n"]+) "', r'"\1"'),  # remove space before and after quotation
            'remove_space_before_punctuation': (" ([" + self.punctuation_marks['after'] + "])", r"\1"),  # remove space before punctuation
            'remove_space_after_punctuation': ("([" + self.punctuation_marks['before'] + "]) ", r"\1"),  # remove space after punctuation
            'add_space_after_period_colon': (
                "([" + self.punctuation_marks['after'][:3] + "])([^ " + self.punctuation_marks['after'] + r"\d۰۱۲۳۴۵۶۷۸۹])",
                r"\1 \2"
            ),  # put space after . and :
            'add_space_after_other_punctuation': (
                "([" + self.punctuation_marks['after'][3:] + "])([^ " + self.punctuation_marks['after'] + "])",
                r"\1 \2"
            ),  # put space after other specified punctuation
            'add_space_before_punctuation': (
                "([^ " + self.punctuation_marks['before'] + "])([" + self.punctuation_marks['before'] + "])",
                r"\1 \2"
            ),  # put space before punctuation
            'add_space_after_number_letters': (
                r"(\d)([آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی])",
                r"\1 \2"
            ),  # put space after numbers before letters
            'add_space_before_number_letters': (
                r"([آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی])(\d)",
                r"\1 \2"
            )  # put space before numbers after letters     
        }
        
        
        self.mi_pattern = r"\bن?می[آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی]+"
    
        self.number_map = {
            'not_persian': "0123456789%٠١٢٣٤٥٦٧٨٩",
            'persian': "۰۱۲۳۴۵۶۷۸۹٪۰۱۲۳۴۵۶۷۸۹"
        }
        self.arabic_replacements = {
            "[\u064b\u064c\u064d\u064e\u064f\u0650\u0651\u0652]": "",  # Arabic diacritics -remove FATHATAN, DAMMATAN, KASRATAN, FATHA, DAMMA, KASRA, SHADDA, SUKUN
            "[ك]": "ک",
            "[ي]": "ی",
            "[هٔ]": "ه",
            "[أ]": "ا"
        }
        self.unicode_replacements = {
            "﷽": " بسم الله الرحمن الرحیم ",
            "﷼": " ریال",
            "(ﷰ|ﷹ)": " صلی ",
            "ﷲ": " الله",
            "ﷳ": " اکبر",
            "ﷴ": " محمد",
            "ﷵ": " صلعم",
            "ﷶ": " رسول",
            "ﷷ": " علیه",
            "ﷸ": " وسلم",
            "ﻵ|ﻶ|ﻷ|ﻸ|ﻹ|ﻺ|ﻻ|ﻼ": " لا"
        }
        self.more_than_two_repeat_pattern = (
                r"([آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی])\1{2,}"
            )
        self.repeated_chars_pattern = (
                r"[آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی]*"
                + self.more_than_two_repeat_pattern
                + "[آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی]*"
            )
        
        
    def load_verb_data(self):
        with open('verbs.dat', 'r', encoding='utf-8') as file:
            verbs = [line.strip() for line in file if line.strip()]
            self.present_bons = {verb.split('#')[0].strip() for verb in verbs if '#' in verb}
            self.past_bons = {verb.split('#')[1].strip() for verb in verbs if '#' in verb}
    
    def decrease_repeated_chars(self: "Normalizer", text: str) -> str:
        """

        Examples:
            >>> normalizer = Normalizer()
            >>> normalizer.decrease_repeated_chars('سلامممم به همه')
            'سلام به همه'
        """
        matches = re.finditer(self.repeated_chars_pattern, text)
        for m in matches:
            print(m)
            print("--------")
            word = m.group()
            words = tokenize(text)
            if word in words:
                no_repeat = re.sub(self.more_than_two_repeat_pattern, r"\1", word)
                two_repeat = re.sub(self.more_than_two_repeat_pattern, r"\1\1", word)

                if (no_repeat in words) != (two_repeat in words):
                    r = no_repeat if no_repeat in words else two_repeat
                    text = text.replace(word, r)
                else:
                    text = text.replace(word, two_repeat)

        return text 
    
    def regex_replace(self, pattern_dict, text):
        for pattern, replacement in pattern_dict.items():
            text = re.sub(pattern, replacement, text)
        return text

    def correct_spacing(self, text):
        for pattern_name, item in self.spacing_patterns.items():
            text = re.sub(item[0], item[1], text)
        return text

    def translate_numbers(self, text):
        translation_table = str.maketrans(self.number_map['not_persian'], self.number_map['persian'])
        return text.translate(translation_table)
    
    def seperate_mi(self, text):
        matches = re.findall(self.mi_pattern, text)
        for m in matches:
            without_mi = re.sub("^(ن?می)", "", m)
            if without_mi in self.present_bons or self.past_bons:
                    corrected_format_text = re.sub("^(ن?می)", r"\1‌", m)
                    text = text.replace(m, corrected_format_text)
        return text

    def remove_punc_marks(self, text):
        for pattern_name, pattern in self.punctuation_marks.items():
            text = re.sub(pattern, "", text)
        return text
        
    def normalize(self, text):
        text= self.remove_punc_marks(text)
        text = self.seperate_mi(text)
        text = self.regex_replace(self.arabic_replacements, text)
        text = self.regex_replace(self.unicode_replacements, text)
        text = self.translate_numbers(text)
        text = self.correct_spacing(text)
        #--- text = self.decrease_repeated_chars(text)
        return text

In [20]:
normalizer = TextNormalizer()
text = news['45']['content']
normalized_text = normalizer.normalize(text)
print(normalized_text)


  به گزارش خبرنگار ورزشی فارس بعد از برکناری ناظم الشریعه سرمربی تیم ملی فوتسال وحید شمسایی به عنوان سکاندار جدید  این تیم معرفی شد اما بعد از عزل عزیزی خادم ریاست فدراسیون فوتبال  قرارداد وی و کادر فنی  از سوی فدراسیون مورد تایید قرار نگرفت  در نهایت این قرارداد در جلسه هیات رئیسه در مشهد مطرح و مورد تصویب اعضا قرار گرفت و بر همین اساس شب گذشته این قرارداد از سوی ذیحسابی و اعضای هیات رییسه فدراسیون نهایی و  جهت ابلاغ به سرمربی تیم ملی ارسال شد انتهای پیام


In [21]:
def delete_punctuations(text):
    return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

In [22]:
def stemming(tokens):
        stemmed = []
        stemmer = parsivar.FindStems()
        for token in tokens:
            stemmed.append(stemmer.convert_to_stem(token))
        return stemmed
text_tokens= tokenize(text)
print("Before Stemming tokens:",text_tokens)
text_tokens_stemmed  = stemming(text_tokens)
print("After Stemming tokens",text_tokens_stemmed)

Before Stemming tokens: ['به', 'گزارش', 'خبرنگار', 'ورزشی', 'فارس', 'بعد', 'از', 'برکناری', 'ناظم', 'الشریعه', 'سرمربی', 'تیم', 'ملی', 'فوتسال', 'وحید', 'شمسایی', 'به', 'عنوان', 'سکاندار', 'جدید', 'این', 'تیم', 'معرفی', 'شد', 'اما', 'بعد', 'از', 'عزل', 'عزیزی', 'خادم', 'ریاست', 'فدراسیون', 'فوتبال', 'قرارداد', 'وی', 'و', 'کادر', 'فنی', 'از', 'سوی', 'فدراسیون', 'مورد', 'تایید', 'قرار', 'نگرفت', 'در', 'نهایت', 'این', 'قرارداد', 'در', 'جلسه', 'هیات', 'رئیسه', 'در', 'مشهد', 'مطرح', 'و', 'مورد', 'تصویب', 'اعضا', 'قرار', 'گرفت', 'و', 'بر', 'همین', 'اساس', 'شب', 'گذشته', 'این', 'قرارداد', 'از', 'سوی', 'ذیحسابی', 'و', 'اعضای', 'هیات', 'رییسه', 'فدراسیون', 'نهایی', 'و', 'جهت', 'ابلاغ', 'به', 'سرمربی', 'تیم', 'ملی', 'ارسال', 'شد', 'انتهای', 'پیام']
After Stemming tokens ['به', 'گزارش', 'خبرنگار', 'ورزشی', 'فارس', 'بعد', 'از', 'برکناری', 'ناظم', 'الشریعه', 'سرمربی', 'تیم', 'ملی', 'فوتسال', 'وحید', 'شمسایی', 'به', 'عنوان', 'سکاندار', 'جدید', 'این', 'تیم', 'معرفی', 'شد', 'اما', 'بعد', 'از', 'عزل', 

In [23]:
def top_k_frequent(tokens, k):
    token_counts = Counter(tokens)
    most_common_tokens = token_counts.most_common(k)
    return most_common_tokens

In [24]:

total_tokens = []
counter = 0
for id,data in news.items():
    text = data['content']
    text = delete_punctuations(text)
    text = normalizer.normalize(text)
    tokens = tokenize(text)
    stemmed_tokens = stemming(tokens)
    data['stemmed_tokens'] = stemmed_tokens
    total_tokens+= stemmed_tokens
    counter += 1
    if counter % 300 == 0:
        print(counter, 'docs processed')
 
    
   


300 docs processed
600 docs processed
900 docs processed
1200 docs processed
1500 docs processed
1800 docs processed
2100 docs processed
2400 docs processed
2700 docs processed
3000 docs processed
3300 docs processed
3600 docs processed
3900 docs processed
4200 docs processed
4500 docs processed
4800 docs processed
5100 docs processed
5400 docs processed
5700 docs processed
6000 docs processed
6300 docs processed
6600 docs processed
6900 docs processed
7200 docs processed
7500 docs processed
7800 docs processed
8100 docs processed
8400 docs processed
8700 docs processed
9000 docs processed
9300 docs processed
9600 docs processed
9900 docs processed
10200 docs processed
10500 docs processed
10800 docs processed
11100 docs processed
11400 docs processed
11700 docs processed
12000 docs processed


In [25]:
top_50 = top_k_frequent(total_tokens,50)
top_50_tks = [x[0] for x in top_50]
print(top_50)
# for count, tk in top_50:
#     print(count,tk)
    # print("-----")

[('و', 219205), ('در', 164329), ('به', 133276), ('از', 92929), ('این', 82976), ('که', 76240), ('با', 68992), ('را', 67489), ('اس', 58060), ('کرد&کن', 45105), ('برای', 30996), ('داشت&دار', 30051), ('شد&شو', 29280), ('تیم', 27692), ('کرد', 22936), ('بود&باش', 22489), ('هم', 22386), ('کشور', 21730), ('ما', 19715), ('یک', 18733), ('بازی', 17796), ('آن', 16860), ('شد', 16595), ('باید', 16196), ('تا', 15870), ('بر', 15685), ('وی', 15483), ('داد&ده', 14849), ('شده', 14578), ('خود', 14552), ('مجلس', 14520), ('اسلامی', 14416), ('گزارش', 14040), ('فارس', 13969), ('گفت', 13773), ('مردم', 13201), ('پیام', 13112), ('ایران', 12717), ('خبرگزاری', 12429), ('انتهای', 12250), ('دولت', 12236), ('اما', 12105), ('سال', 11954), ('گرفت&گیر', 11845), ('رئیس', 11627), ('توانست&توان', 11059), ('بازیکن', 11023), ('داشت&دارد', 10782), ('ملی', 10235), ('اینکه', 10051)]


In [26]:
for id,data in news.items():          
    data['stemmed_tokens'] = [token for token in data['stemmed_tokens'] if token not in top_50_tks]
         

Deleted top 50 tokens for doc 45: ['به', 'گزارش', 'فارس', 'از', 'تیم', 'ملی', 'این', 'شد', 'اما', 'وی', 'و', 'گرفت&گیر', 'در', 'بر', 'انتهای', 'پیام']


## Add new News and preprocessings

### Positional Index

In [32]:
dictionary = {}
def create_postings_list():
    for id,data in news.items():
        for pos,token in enumerate(data['stemmed_tokens']):
            if token not in dictionary:
                dictionary[token]= {}
                dictionary[token]['docs'] = {}
                dictionary[token]['freq'] = 1
                dictionary[token]['docs'][str(id)]= {}
                dictionary[token]['docs'][str(id)]['positions']=[]
                dictionary[token]['docs'][str(id)]['positions'].append(pos)
            else: 
                    if str(id) in dictionary[token]['docs']:
                        dictionary[token]['docs'][str(id)]['positions'].append(pos)
                        dictionary[token]['freq'] +=1
                    else:    
                        dictionary[token]['docs'][str(id)]= {}
                        dictionary[token]['docs'][str(id)]['positions']=[]
                        dictionary[token]['docs'][str(id)]['positions'].append(pos) 
                        dictionary[token]['freq']+=1        
                
create_postings_list()

for term in dictionary:
    for doc_id in dictionary[term]['docs']:
        dictionary[term]['docs'][doc_id]['tf'] = len(dictionary[term]['docs'][doc_id]['positions'])
        
        
for key,value in dictionary.items():
    print(key)
    print(value)
    break

کنفدراسیون
{'docs': {'0': {'positions': [0], 'tf': 1}, '8': {'positions': [178], 'tf': 1}, '34': {'positions': [98], 'tf': 1}, '202': {'positions': [89], 'tf': 1}, '305': {'positions': [18], 'tf': 1}, '387': {'positions': [41], 'tf': 1}, '409': {'positions': [1, 47], 'tf': 2}, '482': {'positions': [148], 'tf': 1}, '483': {'positions': [1, 58], 'tf': 2}, '495': {'positions': [119], 'tf': 1}, '496': {'positions': [5, 38], 'tf': 2}, '502': {'positions': [22], 'tf': 1}, '561': {'positions': [0], 'tf': 1}, '768': {'positions': [2], 'tf': 1}, '798': {'positions': [63], 'tf': 1}, '884': {'positions': [1, 25, 82], 'tf': 3}, '893': {'positions': [0], 'tf': 1}, '897': {'positions': [0], 'tf': 1}, '947': {'positions': [289], 'tf': 1}, '1010': {'positions': [3], 'tf': 1}, '1105': {'positions': [69, 89, 176, 189], 'tf': 4}, '1199': {'positions': [24], 'tf': 1}, '1201': {'positions': [44, 85], 'tf': 2}, '1271': {'positions': [48], 'tf': 1}, '1308': {'positions': [2, 222], 'tf': 2}, '1315': {'positio

## tf-idf vectors (with champions list)

In [43]:
doc_vectors = {} # doc_id : { terms: tf-idf}
N = len(news)
for term in dictionary:
    df_t = len(dictionary[term]['docs'])
    for doc_id in dictionary[term]['docs']:
        if doc_id not in doc_vectors:
            doc_vectors[doc_id]= {}
        tf_value= dictionary[term]['docs'][doc_id]['tf']
        tf_idf = (1+np.log10(tf_value)) * np.log10(N/df_t)
        doc_vectors[doc_id][term] = tf_idf
        dictionary[term]['docs'][doc_id]['tf-idf'] = tf_idf
        
    dictionary[term]['chmp_list'] = sorted(dictionary[term]['docs'],key =lambda doc: dictionary[term]['docs'][doc]['tf'],reverse=True)
    

In [44]:
doc_vectors['12132']['دانشگاه']

2.0323139750307972

In [45]:
dictionary['دانشگاه']['docs']
    

{'105': {'positions': [218, 228, 238], 'tf': 3, 'tf-idf': 1.6882558039616187},
 '154': {'positions': [836, 930, 1085], 'tf': 3, 'tf-idf': 1.6882558039616187},
 '452': {'positions': [237], 'tf': 1, 'tf-idf': 1.1429365047502662},
 '974': {'positions': [28, 54, 90], 'tf': 3, 'tf-idf': 1.6882558039616187},
 '1073': {'positions': [33, 78], 'tf': 2, 'tf-idf': 1.4869946758194448},
 '1307': {'positions': [38], 'tf': 1, 'tf-idf': 1.1429365047502662},
 '1403': {'positions': [55], 'tf': 1, 'tf-idf': 1.1429365047502662},
 '1422': {'positions': [154], 'tf': 1, 'tf-idf': 1.1429365047502662},
 '1435': {'positions': [270], 'tf': 1, 'tf-idf': 1.1429365047502662},
 '1603': {'positions': [154], 'tf': 1, 'tf-idf': 1.1429365047502662},
 '1810': {'positions': [16], 'tf': 1, 'tf-idf': 1.1429365047502662},
 '1950': {'positions': [233], 'tf': 1, 'tf-idf': 1.1429365047502662},
 '2355': {'positions': [10, 22], 'tf': 2, 'tf-idf': 1.4869946758194448},
 '2358': {'positions': [7, 67, 71, 154, 158],
  'tf': 5,
  'tf-

## Only tf Positional Index

## Query Scoring

In [46]:
def query_preprocess(query):
    query = delete_punctuations(query)
    query = normalizer.normalize(query)
    query_tokens = tokenize(query)
    stemmed_query_tokens = stemming(query_tokens)
    return stemmed_query_tokens

def doc_vector_length(doc_id):
    counter = 0
    for term in doc_vectors[doc_id]:
        counter += np.power(doc_vectors[doc_id][term],2)
    return np.sqrt(counter)

def cosine_score(query, k):
    scores = {} #doc_id : score
    q_tokens = query_preprocess(query)
    count = Counter(q_tokens)
    for q_term in q_tokens:
        w_tq = (1+np.log10(count[q_term])) 
        for doc in dictionary[q_term]['chmp_list']:
            w_td = dictionary[q_term]['docs'][doc]['tf-idf']
            if doc in scores:
                scores[doc] += w_tq * w_td
            else:
                scores[doc] = w_tq * w_td
    normalized_scores = {}
    for d in scores:
        normalized_scores[d] = scores[d] / doc_vector_length(d)
    sorted_scores = sorted(normalized_scores.items(), key=lambda x:x[1], reverse=True)
    return sorted_scores[:k]


In [53]:
results = cosine_score("تفحص شستا",3)
for rank,(doc_id , score) in  enumerate(results):
    print(news[doc_id]['title'])
    print(news[doc_id]['url'])
    print('Rank',rank)
    print('Cosine Score:', score)
    print("-----------------")


تحقیق و تفحص از «شستا» و «صندوق بازنشستگی صداوسیما» در کمیسیون اجتماعی تصویب شد
https://www.farsnews.ir/news/14000818000805/تحقیق-و-تفحص-از-شستا-و-صندوق-بازنشستگی-صداوسیما-در-کمیسیون-اجتماعی
Rank 0
Cosine Score: 0.5449146749670559
-----------------
قرائت گزارش در مجلس درباره عملکرد بودجه سال ۱۳۹۹
https://www.farsnews.ir/news/14000923000057/قرائت-گزارش-در-مجلس-درباره-عملکرد-بودجه-سال-۱۳۹۹
Rank 1
Cosine Score: 0.4990358660017677
-----------------
لایحه بودجه حوالی 20 بهمن به صحن مجلس می‌آید
https://www.farsnews.ir/news/14000921000754/لایحه-بودجه-حوالی-20-بهمن-به-صحن-مجلس-می‌آید
Rank 2
Cosine Score: 0.394031687493867
-----------------
